In [1]:
# Download latest version of GRAZPEDWRI-DX Dataset (16GB)
import kagglehub
path = kagglehub.dataset_download("jasonroggy/grazpedwri-dx")
!mv "{path}" /content/

100%|██████████| 15.1G/15.1G [12:12<00:00, 22.2MB/s]

Extracting files...


In [2]:
# Move and Merge all the images to 'dataset/images' folder
!mkdir -p "/content/dataset/images"
!mv /content/1/images_part*/** /content/dataset/images/

In [3]:
# Move all the labels for the images to 'dataset/labels' folder
!mkdir -p "/content/dataset/labels"
!mv /content/1/folder_structure/yolov5/labels/* /content/dataset/labels/

In [4]:
# Move the meta.yaml file to 'dataset/' as well
!mv "/content/1/folder_structure/yolov5/meta.yaml" "/content/dataset/"

# Remove all the previous content and Paste the following into it
# path: /content/dataset

# train: images/train
# val: images/val
# test: images/test

# nc: 9
# names:
#   - boneanomaly
#   - bonelesion
#   - foreignbody
#   - fracture
#   - metal
#   - periostealreaction
#   - pronatorsign
#   - softtissue
#   - text

In [5]:
# Organize the dataset into the following format for YOLLO
# dataset/
#  ├── images/
#  │   ├── train/
#  │   ├── val/
#  │   └── test/
#  ├── labels/
#  │   ├── train/
#  │   ├── val/
#  │   └── test/
#  └── meta.yaml

import os
import random
import shutil
from pathlib import Path

# Paths
dataset_path = Path("/content/dataset")
images_path = dataset_path / "images"
labels_path = dataset_path / "labels"

# Target dirs
for split in ["train", "val", "test"]:
    (dataset_path / "images" / split).mkdir(parents=True, exist_ok=True)
    (dataset_path / "labels" / split).mkdir(parents=True, exist_ok=True)

# Collect all image files
image_files = [f for f in images_path.glob("*.png")] + [f for f in images_path.glob("*.jpg")]
random.shuffle(image_files)

# Split sizes
n = len(image_files)
train_split = int(0.7 * n)
val_split = int(0.9 * n)

splits = {
    "train": image_files[:train_split],
    "val": image_files[train_split:val_split],
    "test": image_files[val_split:]
}

# Move files
for split, files in splits.items():
    for img_file in files:
        # Image
        shutil.move(str(img_file), dataset_path / "images" / split / img_file.name)

        # Label (same name, .txt extension)
        label_file = labels_path / (img_file.stem + ".txt")
        if label_file.exists():
            shutil.move(str(label_file), dataset_path / "labels" / split / label_file.name)

print("✅ Dataset successfully split into train/val/test")


✅ Dataset successfully split into train/val/test


In [ ]:
!pip install -U ultralytics

In [7]:
# Download + load YOLOv11 medium pretrained weights
from ultralytics import YOLO
# model = YOLO("yolo11m.pt") # Medium
model = YOLO("yolo11x.pt") # Large

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
# Training
model.train(
    data="/content/dataset/meta.yaml",
    epochs=2,
    imgsz=512,
    batch=4
)

Ultralytics 8.3.195 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/meta.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tr

KeyboardInterrupt: 

In [9]:
# inference (test on new images)
# You don’t need .txt or .json for testing — those are only for training annotations
# For inference, you just give an image, and YOLO will return bounding boxes + labels

results = model.predict("/content/dataset/images/test", save=True, conf=0.25)


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2033 /content/dataset/images/test/0003_0663715732_02_WRI-R1_M011.png: 640x448 2 fractures, 1 text, 79.4ms
image 2/2033 /content/dataset/images/test/0003_0663715782_02_WRI-R2_M011.png: 640x352 1 fracture, 1 text, 86.8ms
image 3/2033 /content/dataset/images/test/0008_1128992441_01_WRI-L2_M005.png: 640x288 2 fractures, 1 text, 103.7ms
image 4/2033 /content/dataset/images/test/0014_0516793680_02_WRI-L1_M006.png: 640x480 1 fracture, 2 texts, 85.9ms
